# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

2023-10-18 11:06:41.671900: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 11:06:42.188777: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-18 11:06:42.188797: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-18 11:06:43.506304: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# prepare our data and preprocess it

In [2]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /=255.
    return image,label

scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

buffer_size = 10000
shuffle_train_validation_data = scaled_train_validation_data.shuffle(buffer_size)

validation_data = shuffle_train_validation_data.take(num_validation_samples)
train_data = shuffle_train_validation_data.skip(num_validation_samples)

batch_size = 32
train_data = train_data.batch(batch_size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2023-10-14 10:12:06.954036: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-10-14 10:12:06.955199: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/abdelmoneim/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-10-14 10:12:06.955675: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/abdelmoneim/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-10-14 10:12:06.955859: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64

## outline model

In [3]:
input_size = 784
output_size = 10
hidden_layers = 128

model = tf.keras.Sequential([
                                #convloution step
                                tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28,28,1]),
                                
                                #pooling step
                                tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2),
                
                                #adding a second convolutional layer
                                tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28,28,1]),
                                tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2),
    
                                #adding a third convolutional layer
                                tf.keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu', input_shape = [28,28,1]),
                                tf.keras.layers.MaxPool2D(pool_size = 2, strides = 2),
    
                                #flattening
                                tf.keras.layers.Flatten(),
    
                                #full connection
                                tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                                #output layer
                                tf.keras.layers.Dense(output_size, activation = 'softmax')
])

In [4]:
model.compile(optimizer = 'adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

## training

In [5]:
model.fit(train_data,epochs = 10,validation_data = (validation_inputs,validation_targets),verbose = 2)

Epoch 1/10
1688/1688 - 30s - loss: 0.2907 - accuracy: 0.9087 - val_loss: 0.1330 - val_accuracy: 0.9605 - 30s/epoch - 18ms/step
Epoch 2/10
1688/1688 - 25s - loss: 0.1054 - accuracy: 0.9676 - val_loss: 0.0863 - val_accuracy: 0.9728 - 25s/epoch - 15ms/step
Epoch 3/10
1688/1688 - 28s - loss: 0.0764 - accuracy: 0.9763 - val_loss: 0.0621 - val_accuracy: 0.9808 - 28s/epoch - 17ms/step
Epoch 4/10
1688/1688 - 29s - loss: 0.0632 - accuracy: 0.9805 - val_loss: 0.0646 - val_accuracy: 0.9798 - 29s/epoch - 17ms/step
Epoch 5/10
1688/1688 - 28s - loss: 0.0528 - accuracy: 0.9833 - val_loss: 0.0491 - val_accuracy: 0.9855 - 28s/epoch - 17ms/step
Epoch 6/10
1688/1688 - 27s - loss: 0.0447 - accuracy: 0.9866 - val_loss: 0.0454 - val_accuracy: 0.9865 - 27s/epoch - 16ms/step
Epoch 7/10
1688/1688 - 29s - loss: 0.0384 - accuracy: 0.9876 - val_loss: 0.0368 - val_accuracy: 0.9883 - 29s/epoch - 17ms/step
Epoch 8/10
1688/1688 - 28s - loss: 0.0340 - accuracy: 0.9891 - val_loss: 0.0314 - val_accuracy: 0.9907 - 28s/ep

## testing

In [6]:
test_loss, test_accuracy = model.evaluate(test_data)

2023-10-14 10:17:09.788209: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 865280000 exceeds 10% of free system memory.


1/1 [==============================] - 1s 863ms/step - loss: 0.0438 - accuracy: 0.9879


In [7]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.04. Test accuracy: 98.79%


**Thank you**